In [36]:
# Modules we need to import to obtain our data
import pandas as pd
import numpy as np
import matplotlib
import requests
import json
from tqdm import tqdm 
from pymongo import MongoClient
import os


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-  
**Requirements:**  
1. Data must be stored in and extracted from at least one database (PostgreSQL, MongoDB, SQLite, etc).
2. Your project must include at least one JavaScript OR Python library that we did not cover.
3. Your project must be powered by a dataset with at least 100 records.  
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [37]:
#Set up your API request and URL
base_url = "http://apps.who.int/gho/athena/api/GHO/"

#To obtain the DTP3-containing-vaccine among 1 year olds (%)
vacc_rates_code =  "WHS4_100"
full_vacc_rates_url = f'{base_url}{vacc_rates_code}'

#To obtain the percentage of MCV1 Coverage among 1 year olds
meas_vacc_rates_code = "WHS8_110"
meas_vacc_rates_url = f'{base_url}{meas_vacc_rates_code}'

#To obtain the Pertussis incidence rate for each country
pertussis_incidence_code = "WHS3_43"
pertussis_incidence_url = f'{base_url}{pertussis_incidence_code}'

#To obtain the neonatal tetanus incidence rate for each country
tetanus_incidence_code = "WHS3_56"
tetanus_incidence_url = f'{base_url}{tetanus_incidence_code}'

#To obtain the measles incidence rate for each country
measles_incidence_code = "WHS3_62"
measles_incidence_url = f'{base_url}{measles_incidence_code}'

#To obtain the total number of diphteria cases for each country
diphtheria_incidence_code = "WHS3_41"
diphtheria_incidence_url = f'{base_url}{diphtheria_incidence_code}'

#To obtain the Deaths in children aged <5 years, by cause
endpoint_measles_mortality_url = "https://www.who.int/data/gho/indicator-metadata-registry/imr-details/89"
dtp_mortality_code = "CM_01"
dtp_mortality_url = f'{base_url}{dtp_mortality_code}'

#To define which countries and years we're trying to study
countries = []
years_list = list(range(2008, 2021))

#Let's check the URLs
print(full_vacc_rates_url)
print(pertussis_incidence_url)
print(dtp_mortality_url)

http://apps.who.int/gho/athena/api/GHO/WHS4_100
http://apps.who.int/gho/athena/api/GHO/WHS3_43
http://apps.who.int/gho/athena/api/GHO/CM_01


The queries (without a filter) should read: 
1. http://apps.who.int/GHO/MCV2/
2. http://apps.who.int/GHO/WHS3_62
3. http://apps.who.int/GHO/WHS2_168

In [38]:
# Let's assign a list of country codes to each country
country_codes = "/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/country-codes-WHO.csv" 
df_country_codes = pd.read_csv(country_codes)

# AI USE: We obtained this list of countries by using a ChatGPT Prompt
american_countries = [
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", 
    "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname", 
    "Uruguay", "Venezuela", "Antigua and Barbuda", "Bahamas", 
    "Barbados", "Belize", "Canada", "Costa Rica", "Cuba", 
    "Dominica", "Dominican Republic", "El Salvador", "Grenada", 
    "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", 
    "Nicaragua", "Panama", "Saint Kitts and Nevis", "Saint Lucia", 
    "Saint Vincent and the Grenadines", "Trinidad and Tobago", 
    "United States of America"
]

american_country_codes = {row["COUNTRY"]: row["CODE"] for _, row in df_country_codes.iterrows() if row["COUNTRY"] in american_countries}
print(american_country_codes)

query_codes = list(american_country_codes.values())
        



{'Antigua and Barbuda': 'ATG', 'Argentina': 'ARG', 'Bahamas': 'BHS', 'Barbados': 'BRB', 'Belize': 'BLZ', 'Bolivia': 'BOL', 'Brazil': 'BRA', 'Canada': 'CAN', 'Chile': 'CHL', 'Colombia': 'COL', 'Costa Rica': 'CRI', 'Cuba': 'CUB', 'Dominica': 'DMA', 'Dominican Republic': 'DOM', 'Ecuador': 'ECU', 'El Salvador': 'SLV', 'Grenada': 'GRD', 'Guatemala': 'GTM', 'Guyana': 'GUY', 'Haiti': 'HTI', 'Honduras': 'HND', 'Jamaica': 'JAM', 'Mexico': 'MEX', 'Nicaragua': 'NIC', 'Panama': 'PAN', 'Paraguay': 'PRY', 'Peru': 'PER', 'Saint Kitts and Nevis': 'KNA', 'Saint Lucia': 'LCA', 'Saint Vincent and the Grenadines': 'VCT', 'Suriname': 'SUR', 'Trinidad and Tobago': 'TTO', 'United States of America': 'USA', 'Uruguay': 'URY', 'Venezuela': 'VEN'}


In [39]:
#In order to filter by country, we need to add "?filter=COUNTRY:COUNTRY_NAME" to the query.
filter = "?filter="
country_name = ""
filter_country = f"?filter=SpatialDimType:COUNTRY,SpatialDim={{country_code}}"
filter_divider = ";"
year = ""
filter_year = f";TimeDimType:YEAR,TimeDim={{year}}"

In [40]:
#To obtain country codes.

country_code_url = "https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues"

response = requests.get(country_code_url)

if response.status_code == 200:  
    data = response.json()  
    print("Response Keys:", data.keys())  

if "value" in data:
    country_codes = [entry["Code"] for entry in data["value"] if "Code" in entry]
    print("Retrieved Country Codes:", country_codes)
  
    for print_response in data:  
        sentence = f'Status: RETRIEVED'
        print(sentence)  
else:  
    sentence = f'Status: NOT RETRIEVED'  
    print(sentence)  
response

Response Keys: dict_keys(['@odata.context', 'value'])
Retrieved Country Codes: ['ABW', 'AFG', 'AGO', 'AIA', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BES', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHI', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FLK', 'FRA', 'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GLP', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUF', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAC', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'ME1', 'ME

<Response [200]>

In [41]:
# Pertussis incidence database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Pertussis_Incidence/Pertussis_Incidence_{name}.csv'
    
    if country_code is None:
        tqdm.write(f'Country code for {name} was not found')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect')

    #Query we're going to use.
    retrieve_query = f"{pertussis_incidence_url}.csv?filter=COUNTRY:{country_code}"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Pertussis_Incidence/Pertussis_Incidence_{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'Pertussis incidence rate for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:01<01:07,  1.98s/it]

Pertussis incidence rate for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:03<00:49,  1.49s/it]

...

Pertussis incidence rate for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:35<00:00,  1.04it/s]

Pertussis incidence rate for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:36<00:00,  1.04s/it]

Pertussis incidence rate for United States of America was downloaded succesfully!


In [42]:
# Tetanus incidence database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Tetanus_Incidence/Tetanus_Incidence_{name}.csv'
    
    if country_code is None:
        tqdm.write(f'Country code for {name} was not found')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect')

    #Query we're going to use.
    retrieve_query = f"{tetanus_incidence_url}.csv?filter=COUNTRY:{country_code}"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Tetanus_Incidence/Tetanus_Incidence_{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'Tetanus incidence rate for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:00<00:32,  1.04it/s]

Tetanus incidence rate for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:01<00:31,  1.04it/s]

...

Tetanus incidence rate for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:32<00:00,  1.02it/s]

Tetanus incidence rate for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:33<00:00,  1.03it/s]

Tetanus incidence rate for United States of America was downloaded succesfully!


In [43]:
# Diphtheria incidence database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Diphtheria_Incidence/Diphtheria_Incidence_{name}.csv'
    
    if country_code is None:
        tqdm.write(f'Country code for {name} was not found')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect')

    #Query we're going to use.
    retrieve_query = f"{diphtheria_incidence_url}.csv?filter=COUNTRY:{country_code}"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Diphtheria_Incidence/Diphtheria_Incidence_{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'Diphtheria incidence rate for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:00<00:32,  1.05it/s]

Diphtheria incidence rate for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:01<00:31,  1.05it/s]

...

Diphtheria incidence rate for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:34<00:00,  1.04it/s]

Diphtheria incidence rate for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:35<00:00,  1.02s/it]

Diphtheria incidence rate for United States of America was downloaded succesfully!


In [44]:
# Measles incidence database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Measles_Incidence/Measles_Incidence{name}.csv'
    
    if country_code is None:
        tqdm.write(f'Country code for {name} was not found')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect')

    #Query we're going to use.
    retrieve_query = f"{measles_incidence_url}.csv?filter=COUNTRY:{country_code}"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Measles_Incidence/Measles_Incidence{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'Measles incidence rate for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:00<00:32,  1.05it/s]

Measles incidence rate for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:01<00:31,  1.05it/s]

...

Measles incidence rate for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:34<00:01,  1.02s/it]

Measles incidence rate for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:35<00:00,  1.02s/it]

Measles incidence rate for United States of America was downloaded succesfully!


In [45]:
# Mortality rate <5 database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Child_Mortality/Child_Mortality_{name}.csv'
    
    if country_code is None:
        tqdm.write(f'❗❗❗ Country code for {name} was not found❗❗❗')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect❗')

    #Query we're going to use.
    retrieve_query = f"{dtp_mortality_url}.csv?filter=COUNTRY:{country_code};SEX:BTSX"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Child_Mortality/Child_Mortality_{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'Child mortality file for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:00<00:32,  1.05it/s]

Child mortality file for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:01<00:31,  1.04it/s]

...

Child mortality file for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:36<00:01,  1.08s/it]

Child mortality file for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:37<00:00,  1.06s/it]

Child mortality file for United States of America was downloaded succesfully!


In [46]:
# DTP vaccination rate database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Vaccination_Rate/DTP3_Vacc_Rates_{name}.csv'
    
    if country_code is None:
        tqdm.write(f'❗❗❗ Country code for {name} was not found❗❗❗')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect❗')

    #Query we're going to use.
    retrieve_query = f"{full_vacc_rates_url}.csv?filter=COUNTRY:{country_code}"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Vaccination_Rate/DTP3_Vacc_Rates_{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'DPT3 File for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:00<00:32,  1.03it/s]

DPT3 File for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:01<00:31,  1.03it/s]

...

DPT3 File for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:35<00:00,  1.03it/s]

DPT3 File for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:35<00:00,  1.03s/it]

DPT3 File for United States of America was downloaded succesfully!


In [47]:
# Measles vaccination rate database
for i in tqdm(range(len(american_countries)), desc="Processing Countries", leave=True, position=0):  
    #Let's set the variables we're going to use
    name = american_countries[i]  
    country_code = american_country_codes.get(name)
    output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/M_Vaccination_Rate/Meas_Vacc_Rates_{name}.csv'
    
    if country_code is None:
        tqdm.write(f'❗❗❗ Country code for {name} was not found❗❗❗')
        continue

    if country_code in american_country_codes.values():
        tqdm.write(f'✅ Correct country code for {name} was assigned.')
    else:
        tqdm.write(f'❌ Country code for {name} is incorrect❗')

    #Query we're going to use.
    retrieve_query = f"{meas_vacc_rates_url}.csv?filter=COUNTRY:{country_code}"

    try:
        response = requests.get(retrieve_query, stream = True)
        response.raise_for_status()  

        with open(f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/M_Vaccination_Rate/Meas_Vacc_Rates_{name}.csv', 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk:
                    file.write(chunk)
        print(f'Measles Vacc File for {name} was downloaded succesfully!')
    
    except requests.exceptions.RequestException as e:
        print(f'An error occurred. Check the log: {e}')

Processing Countries:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Correct country code for Argentina was assigned.


Processing Countries:   3%|▎         | 1/35 [00:00<00:32,  1.03it/s]

Measles Vacc File for Argentina was downloaded succesfully!
✅ Correct country code for Bolivia was assigned.


Processing Countries:   6%|▌         | 2/35 [00:01<00:31,  1.04it/s]

...

Measles Vacc File for Saint Vincent and the Grenadines was downloaded succesfully!
✅ Correct country code for Trinidad and Tobago was assigned.


Processing Countries:  97%|█████████▋| 34/35 [00:40<00:01,  1.08s/it]

Measles Vacc File for Trinidad and Tobago was downloaded succesfully!
✅ Correct country code for United States of America was assigned.


Processing Countries: 100%|██████████| 35/35 [00:41<00:00,  1.19s/it]

Measles Vacc File for United States of America was downloaded succesfully!


In [48]:
# MongoDB Connection Setup for Child Mortality
MONGO_URI = "mongodb://localhost:27017/"  
DATABASE_NAME = "proyect3_db"  
COLLECTION_NAME = "mortality_data"  
output_csv = f'/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Child_Mortality/Combined/Combined_Databases.csv'

CSV_FOLDER_PATH = "/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Child_Mortality"  

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

def import_csv_to_mongodb():
    all_data = []

    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"):  
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: Child_Mortality_Barbados.csv - 31 rows
Processed: Child_Mortality_El Salvador.csv - 31 rows
Processed: Child_Mortality_Canada.csv - 31 rows
Processed: Child_Mortality_Chile.csv - 31 rows
Processed: Child_Mortality_Guyana.csv - 31 rows
Processed: Child_Mortality_Guatemala.csv - 31 rows
Processed: Child_Mortality_Antigua and Barbuda.csv - 31 rows
Processed: Child_Mortality_Venezuela.csv - 31 rows
Processed: Child_Mortality_Argentina.csv - 31 rows
Processed: Child_Mortality_Peru.csv - 31 rows
Processed: Child_Mortality_United States of America.csv - 31 rows
Processed: Child_Mortality_Costa Rica.csv - 31 rows
Processed: Child_Mortality_Haiti.csv - 31 rows
Processed: Child_Mortality_Uruguay.csv - 31 rows
Processed: Child_Mortality_Paraguay.csv - 31 rows
Processed: Child_Mortality_Dominica.csv - 31 rows
Processed: Child_Mortality_Mexico.csv - 31 rows
Processed: Child_Mortality_Suriname.csv - 31 rows
Processed: Child_Mortality_Belize.csv - 31 rows
Processed: Child_Mortality_Bahamas

In [49]:
# MongoDB Connection Setup for Measles Incidence
MONGO_URI = "mongodb://localhost:27017/" 
DATABASE_NAME = "proyect3_db"  
COLLECTION_NAME = "measles_inc_data"  

CSV_FOLDER_PATH = '/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Measles_Incidence'  

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

def import_csv_to_mongodb():
    all_data = []

    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"):  
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: Measles_IncidenceSaint Kitts and Nevis.csv - 31 rows
Processed: Measles_IncidenceCuba.csv - 30 rows
Processed: Measles_IncidenceCosta Rica.csv - 30 rows
Processed: Measles_IncidenceAntigua and Barbuda.csv - 31 rows
Processed: Measles_IncidenceUruguay.csv - 31 rows
Processed: Measles_IncidenceGuyana.csv - 31 rows
Processed: Measles_IncidenceEl Salvador.csv - 30 rows
Processed: Measles_IncidenceCanada.csv - 31 rows
Processed: Measles_IncidenceBahamas.csv - 31 rows
Processed: Measles_IncidenceHaiti.csv - 25 rows
Processed: Measles_IncidenceBelize.csv - 31 rows
Processed: Measles_IncidenceMexico.csv - 31 rows
Processed: Measles_IncidenceHonduras.csv - 31 rows
Processed: Measles_IncidenceVenezuela.csv - 31 rows
Processed: Measles_IncidenceArgentina.csv - 31 rows
Processed: Measles_IncidenceChile.csv - 31 rows
Processed: Measles_IncidenceGuatemala.csv - 31 rows
Processed: Measles_IncidenceColombia.csv - 31 rows
Processed: Measles_IncidenceGrenada.csv - 30 rows
Processed: Measles_I

In [50]:
# MongoDB Connection Setup for Measles Vaccination Rates
MONGO_URI = "mongodb://localhost:27017/"  
DATABASE_NAME = "proyect3_db"  
COLLECTION_NAME = "measles_vacc_data"  

CSV_FOLDER_PATH = '/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/M_Vaccination_Rate'

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

def import_csv_to_mongodb():
    all_data = []

    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"): 
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: Meas_Vacc_Rates_Bolivia.csv - 22 rows
Processed: Meas_Vacc_Rates_Jamaica.csv - 22 rows
Processed: Meas_Vacc_Rates_United States of America.csv - 22 rows
Processed: Meas_Vacc_Rates_Mexico.csv - 22 rows
Processed: Meas_Vacc_Rates_Belize.csv - 22 rows
Processed: Meas_Vacc_Rates_Venezuela.csv - 22 rows
Processed: Meas_Vacc_Rates_Paraguay.csv - 22 rows
Processed: Meas_Vacc_Rates_Argentina.csv - 22 rows
Processed: Meas_Vacc_Rates_Guatemala.csv - 22 rows
Processed: Meas_Vacc_Rates_Suriname.csv - 22 rows
Processed: Meas_Vacc_Rates_Dominica.csv - 22 rows
Processed: Meas_Vacc_Rates_Barbados.csv - 22 rows
Processed: Meas_Vacc_Rates_El Salvador.csv - 22 rows
Processed: Meas_Vacc_Rates_Peru.csv - 22 rows
Processed: Meas_Vacc_Rates_Antigua and Barbuda.csv - 22 rows
Processed: Meas_Vacc_Rates_Ecuador.csv - 22 rows
Processed: Meas_Vacc_Rates_Canada.csv - 22 rows
Processed: Meas_Vacc_Rates_Grenada.csv - 22 rows
Processed: Meas_Vacc_Rates_Guyana.csv - 22 rows
Processed: Meas_Vacc_Rates_Saint 

In [51]:
# MongoDB Connection Setup for DTP Vaccination Rates
MONGO_URI = "mongodb://localhost:27017/" 
DATABASE_NAME = "proyect3_db"  
COLLECTION_NAME = "dtp_vacc_data"  

CSV_FOLDER_PATH = '/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Vaccination_Rate'  

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

def import_csv_to_mongodb():
    all_data = []

    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"):  
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: DTP3_Vacc_Rates_United States of America.csv - 22 rows
Processed: DTP3_Vacc_Rates_Dominica.csv - 22 rows
Processed: DTP3_Vacc_Rates_Ecuador.csv - 22 rows
Processed: DTP3_Vacc_Rates_Suriname.csv - 22 rows
Processed: DTP3_Vacc_Rates_Paraguay.csv - 22 rows
Processed: DTP3_Vacc_Rates_Grenada.csv - 22 rows
Processed: DTP3_Vacc_Rates_El Salvador.csv - 22 rows
Processed: DTP3_Vacc_Rates_Trinidad and Tobago.csv - 22 rows
Processed: DTP3_Vacc_Rates_Brazil.csv - 22 rows
Processed: DTP3_Vacc_Rates_Panama.csv - 22 rows
Processed: DTP3_Vacc_Rates_Jamaica.csv - 22 rows
Processed: DTP3_Vacc_Rates_Nicaragua.csv - 22 rows
Processed: DTP3_Vacc_Rates_Dominican Republic.csv - 22 rows
Processed: DTP3_Vacc_Rates_Bolivia.csv - 22 rows
Processed: DTP3_Vacc_Rates_Cuba.csv - 22 rows
Processed: DTP3_Vacc_Rates_Saint Kitts and Nevis.csv - 22 rows
Processed: DTP3_Vacc_Rates_Barbados.csv - 22 rows
Processed: DTP3_Vacc_Rates_Costa Rica.csv - 22 rows
Processed: DTP3_Vacc_Rates_Uruguay.csv - 22 rows
Process

In [52]:
# MongoDB Connection Setup for Pertussis incidence Rates
MONGO_URI = "mongodb://localhost:27017/"  
DATABASE_NAME = "proyect3_db" 
COLLECTION_NAME = "Pert_incidence_data"  

CSV_FOLDER_PATH = '/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Pertussis_Incidence'  

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

def import_csv_to_mongodb():
    all_data = []

    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"): 
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: Pertussis_Incidence_Antigua and Barbuda.csv - 31 rows
Processed: Pertussis_Incidence_Honduras.csv - 31 rows
Processed: Pertussis_Incidence_Ecuador.csv - 30 rows
Processed: Pertussis_Incidence_Grenada.csv - 31 rows
Processed: Pertussis_Incidence_Costa Rica.csv - 31 rows
Processed: Pertussis_Incidence_Saint Vincent and the Grenadines.csv - 31 rows
Processed: Pertussis_Incidence_Saint Kitts and Nevis.csv - 31 rows
Processed: Pertussis_Incidence_Colombia.csv - 31 rows
Processed: Pertussis_Incidence_Haiti.csv - 23 rows
Processed: Pertussis_Incidence_Nicaragua.csv - 31 rows
Processed: Pertussis_Incidence_Panama.csv - 31 rows
Processed: Pertussis_Incidence_Chile.csv - 31 rows
Processed: Pertussis_Incidence_Bolivia.csv - 31 rows
Processed: Pertussis_Incidence_Jamaica.csv - 31 rows
Processed: Pertussis_Incidence_Peru.csv - 31 rows
Processed: Pertussis_Incidence_Brazil.csv - 31 rows
Processed: Pertussis_Incidence_El Salvador.csv - 31 rows
Processed: Pertussis_Incidence_Dominica.csv - 

In [53]:
# MongoDB Connection Setup for Tetanus incidence Rates
MONGO_URI = "mongodb://localhost:27017/" 
DATABASE_NAME = "proyect3_db"  
COLLECTION_NAME = "Tet_incidence_data"  

CSV_FOLDER_PATH = '/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Tetanus_Incidence'  

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

def import_csv_to_mongodb():
    all_data = []

    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"):  
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: Tetanus_Incidence_Haiti.csv - 25 rows
Processed: Tetanus_Incidence_Honduras.csv - 31 rows
Processed: Tetanus_Incidence_Saint Vincent and the Grenadines.csv - 31 rows
Processed: Tetanus_Incidence_Belize.csv - 31 rows
Processed: Tetanus_Incidence_Mexico.csv - 31 rows
Processed: Tetanus_Incidence_Grenada.csv - 31 rows
Processed: Tetanus_Incidence_Ecuador.csv - 30 rows
Processed: Tetanus_Incidence_Colombia.csv - 31 rows
Processed: Tetanus_Incidence_El Salvador.csv - 31 rows
Processed: Tetanus_Incidence_Bolivia.csv - 31 rows
Processed: Tetanus_Incidence_Guatemala.csv - 31 rows
Processed: Tetanus_Incidence_Cuba.csv - 30 rows
Processed: Tetanus_Incidence_Guyana.csv - 31 rows
Processed: Tetanus_Incidence_Canada.csv - 21 rows
Processed: Tetanus_Incidence_Chile.csv - 31 rows
Processed: Tetanus_Incidence_Venezuela.csv - 31 rows
Processed: Tetanus_Incidence_Argentina.csv - 30 rows
Processed: Tetanus_Incidence_Jamaica.csv - 31 rows
Processed: Tetanus_Incidence_Trinidad and Tobago.csv - 3

In [54]:
# MongoDB Connection Setup for Diphtheria incidence Rates
MONGO_URI = "mongodb://localhost:27017/"  
DATABASE_NAME = "proyect3_db"  
COLLECTION_NAME = "Diphth_incidence_data"  

CSV_FOLDER_PATH = '/Users/Kenji-to/Documents/GitHub/Module Challenges/Project 3/Databases_RAW/Diphtheria_Incidence'

client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]


def import_csv_to_mongodb():
    all_data = []


    for file_name in os.listdir(CSV_FOLDER_PATH):
        if file_name.endswith(".csv"): 
            file_path = os.path.join(CSV_FOLDER_PATH, file_name)

            df = pd.read_csv(file_path, usecols=["COUNTRY", "YEAR", "Numeric"])

            # In order to filter this section, we used AI to help troubleshoot the code
            df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")

            df = df[(df["YEAR"] >= 1991) & (df["YEAR"] <= 2021)]

            data_dict = df.to_dict(orient="records")
            all_data.extend(data_dict)

            print(f"Processed: {file_name} - {len(df)} rows")

    if all_data:
        collection.insert_many(all_data)
        print(f"Inserted {len(all_data)} records into MongoDB.")
    else:
        print("No data to insert.")

import_csv_to_mongodb()


Processed: Diphtheria_Incidence_Saint Vincent and the Grenadines.csv - 31 rows
Processed: Diphtheria_Incidence_Peru.csv - 31 rows
Processed: Diphtheria_Incidence_Canada.csv - 31 rows
Processed: Diphtheria_Incidence_Guyana.csv - 31 rows
Processed: Diphtheria_Incidence_Haiti.csv - 27 rows
Processed: Diphtheria_Incidence_Costa Rica.csv - 30 rows
Processed: Diphtheria_Incidence_Jamaica.csv - 31 rows
Processed: Diphtheria_Incidence_Dominica.csv - 31 rows
Processed: Diphtheria_Incidence_Suriname.csv - 31 rows
Processed: Diphtheria_Incidence_Paraguay.csv - 31 rows
Processed: Diphtheria_Incidence_Bolivia.csv - 31 rows
Processed: Diphtheria_Incidence_Trinidad and Tobago.csv - 29 rows
Processed: Diphtheria_Incidence_Chile.csv - 31 rows
Processed: Diphtheria_Incidence_Grenada.csv - 31 rows
Processed: Diphtheria_Incidence_Barbados.csv - 31 rows
Processed: Diphtheria_Incidence_Ecuador.csv - 31 rows
Processed: Diphtheria_Incidence_Mexico.csv - 31 rows
Processed: Diphtheria_Incidence_Belize.csv - 31 

In [ ]:
#List of terminal commands to export all databases as Json files

# mongoexport --db=proyect3_db --collection=diphth_incidence_data --out=diphth_incidence_data.json --jsonArray
# mongoexport --db=proyect3_db --collection=dtp_vacc_data --out=dtp_vacc_data.json --jsonArray
# mongoexport --db=proyect3_db --collection=measles_inc_data --out=measles_inc_data.json --jsonArray
# mongoexport --db=proyect3_db --collection=measles_vacc_data --out=measles_vacc_data.json --jsonArray
# mongoexport --db=proyect3_db --collection=mortality_data --out=mortality_data.json --jsonArray
# mongoexport --db=proyect3_db --collection=Pert_incidence_data --out=Pert_incidence_data.json --jsonArray
# mongoexport --db=proyect3_db --collection=Tet_incidence_data --out=Tet_incidence_data.json --jsonArray



